In [1]:
import torch 

In [2]:
# !pip install sentence_transformers
# !pip install faiss-gpu 

In [3]:
from sentence_transformers import SentenceTransformer
sentences = ["Кошка ловит мышку", "Each sentence is converted"]


from sentence_transformers import SentenceTransformer

model = SentenceTransformer("inkoziev/sbert_pq", device='cuda')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
!ls ../input/data-pkldir/

data.pkl


In [5]:
import pickle
with open('../input/data-pkldir/data.pkl', 'rb') as fp:
    corpus = pickle.load(fp)


In [6]:
import math
batch_size = 100000
num_batches = math.ceil(len(corpus) / batch_size)

In [14]:
num_batches

345

In [7]:
import gc
gc.collect()

37

In [8]:
import faiss
from faiss import write_index, read_index

In [9]:
d = 312
cpu_index = faiss.index_factory(d, "IVF100,PQ8")
cpu_index.is_trained, cpu_index.ntotal
res = faiss.StandardGpuResources()  # use a single GPU

## Using a flat index


# make it a flat GPU index
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, cpu_index)

In [10]:
model.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 2048, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [23]:
with torch.no_grad():
    if True:
        try:
            for i in range(0,200):
                # формируем батч
                start, end = i * batch_size, (i + 1) * batch_size
                corpus_batch = corpus[start:end]

                # считаем вектора для всех предложений в батче
                embeddings = model.encode(
                    corpus_batch,
                    batch_size=5000,
                    show_progress_bar=True
                )

                # добавляем новые батч векторов в индекс и сохраняем его
                gpu_index_flat.train(embeddings)


                print(f'batch: {i + 1} / {num_batches}, vectors: {cpu_index.ntotal}')

                # чистим ОЗУ
                del embeddings
                gc.collect()
                torch.cuda.empty_cache()
        except KeyboardInterrupt:
            print('Остановлено пользователем')
            try:
                del embeddings
            except:
                pass

batch: 175 / 345, vectors: 0


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

batch: 176 / 345, vectors: 0


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

batch: 177 / 345, vectors: 0


...

batch: 198 / 345, vectors: 0


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

batch: 199 / 345, vectors: 0


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

batch: 200 / 345, vectors: 0


In [20]:
cpu_index.is_trained, cpu_index.ntotal

(False, 0)

In [21]:
gpu_index_flat.is_trained, gpu_index_flat.ntotal


(True, 0)

In [27]:

# co = faiss.GpuClonerOptions()
# co.useFloat16 = True
# # co.usePrecomputed = False
# co.indicesOptions = faiss.INDICES_CPU

# res = faiss.StandardGpuResources()

cpu_index = faiss.index_gpu_to_cpu(gpu_index_flat)  # error happens here


In [28]:
cpu_index.is_trained, cpu_index.ntotal

(True, 0)

In [29]:
write_index(cpu_index, 'candidates.index')

In [30]:
!ls -l

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total 436
-rw-r--r-- 1 root root 444468 Nov 24 05:02 candidates.index


In [32]:
!ls -l

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total 31688
-rw-r--r-- 1 root root 32445268 Nov 24 05:09 candidates.index


In [18]:
def get_methods(object, spacing=20):
  methodList = []
  for method_name in dir(object):
    try:
        if callable(getattr(object, method_name)):
            methodList.append(str(method_name))
    except Exception:
        methodList.append(str(method_name))
  processFunc = (lambda s: ' '.join(s.split())) or (lambda s: s)
  for method in methodList:
    try:
        print(str(method.ljust(spacing)) + ' ' +
              processFunc(str(getattr(object, method).__doc__)[0:90]))
    except Exception:
        print(method.ljust(spacing) + ' ' + ' getattr() failed')

get_methods(gpu_index_flat)

__class__            IVFPQ index for the GPU
__delattr__          Implement delattr(self, name).
__dir__              Default dir() implementation.
__eq__               Return self==value.
__format__           Default object formatter.
__ge__               Return self>=value.
__getattribute__     Return getattr(self, name).
__getstate__         None
__gt__               Return self>value.
__hash__             Return hash(self).
__init__             *Overload 1:* Construct from a pre-existing faiss::IndexIVFPQ instance, c
__init_subclass__    This method is called when a class is subclassed. The default implementation does nothing
__le__               Return self<=value.
__lt__               Return self<value.
__ne__               Return self!=value.
__new__              Create and return a new object. See help(type) for accurate signature.
__reduce__           Helper for pickle.
__reduce_ex__        Helper for pickle.
__repr__             None
__setattr__          Implement setattr(sel

In [19]:
get_methods(cpu_index)

__class__            Inverted file with Product Quantizer encoding. Each residual vector is encoded a
__delattr__          Implement delattr(self, name).
__dir__              Default dir() implementation.
__eq__               Return self==value.
__format__           Default object formatter.
__ge__               Return self>=value.
__getattribute__     Return getattr(self, name).
__getstate__         None
__gt__               Return self>value.
__hash__             Return hash(self).
__init__             None
__init_subclass__    This method is called when a class is subclassed. The default implementation does nothing
__le__               Return self<=value.
__lt__               Return self<value.
__ne__               Return self!=value.
__new__              Create and return a new object. See help(type) for accurate signature.
__reduce__           Helper for pickle.
__reduce_ex__        Helper for pickle.
__repr__             None
__setattr__          Implement setattr(self, name, val

In [34]:
del gpu_index_flat

In [35]:
gc.collect()
torch.cuda.empty_cache()

In [37]:
with torch.no_grad():
    if True:
        try:
            for i in range(40,100):
                # формируем батч
                start, end = i * batch_size, (i + 1) * batch_size
                corpus_batch = corpus[start:end]

                # считаем вектора для всех предложений в батче
                embeddings = model.encode(
                    corpus_batch,
                    batch_size=5000,
                    show_progress_bar=True
                )

                # добавляем новые батч векторов в индекс и сохраняем его
                cpu_index.add(embeddings)
                write_index(cpu_index, 'candidates.index')

                print(f'batch: {i + 1} / {num_batches}, vectors: {cpu_index.ntotal}')

                # чистим ОЗУ
                del embeddings
                gc.collect()
                torch.cuda.empty_cache()
        except KeyboardInterrupt:
            print('Остановлено пользователем')
            try:
                del embeddings
            except:
                pass

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

batch: 41 / 345, vectors: 4100000


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

batch: 42 / 345, vectors: 4200000


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

...

batch: 98 / 345, vectors: 9800000


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

batch: 99 / 345, vectors: 9900000


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

batch: 100 / 345, vectors: 10000000


In [38]:
!ls -l

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total 156688
-rw-r--r-- 1 root root 160445268 Nov 24 05:38 candidates.index


In [ ]:
# rm -rf candidates.index